## Prepare train_data and test_data

In [1]:
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

In [2]:
X,y = datasets.load_breast_cancer(as_frame=True,return_X_y=True)
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.7,random_state=335)
train_data = pd.concat([X_train,y_train],axis=1)
eval_data = pd.concat([X_test,y_test],axis=1)

# Use HyperGBM to train model

In [3]:
from hypergbm import HyperGBM
from hypergbm.search_space import search_space_general
from hypernets.searchers.random_searcher import RandomSearcher

In [4]:
# instantiate related objects
searcher = RandomSearcher(search_space_general, optimize_direction='max')
hypergbm = HyperGBM(searcher, task='binary', reward_metric='accuracy')
# search
hypergbm.search(X_train, y_train, X_eval=None, y_eval=None,cv=True, max_trials=20)

In [5]:
# load best model
best_trial = hypergbm.get_best_trial()
estimator = hypergbm.load_estimator(best_trial.model_file)
best_trial

key,value
Trial No.,6
Reward,0.9748743718592965
Elapsed,0.17852544784545898
space.signature,7f41587c0bd6e9402046c943b41e8ef6
space.vectors,"[2, 2, 3, 2, 1, 4]"
0-estimator_options.hp_or,2
1-numeric_imputer_0.strategy,constant
2-numeric_or_scaler_0.hp_or,3
3-Module_CatBoostEstimator_1.learning_rate,0.5
4-Module_CatBoostEstimator_1.depth,5


In [6]:
# Scoring
from hypernets.tabular.metrics import calc_score
preds=estimator.predict(X_test)
proba=estimator.predict_proba(X_test)
scores = calc_score(y_test,preds,proba,task='binary', metrics=['auc','accuracy','f1','recall','precision'] )
scores

{'auc': 0.977113213304852,
 'accuracy': 0.9532163742690059,
 'f1': 0.9652173913043478,
 'recall': 0.9823008849557522,
 'precision': 0.9487179487179487}